# **Implementing Next Word Predictor using LSTM**

In [1]:
!pip install nltk

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from nltk.tokenize import word_tokenize
import nltk

### **1. Data Loading**

### **2. Tokenization**

In [3]:
# Tokenization
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
document = ["""
              Emma Woodhouse, handsome, clever, and rich, with a comfortable home
              and happy disposition, seemed to unite some of the best blessings
              of existence; and had lived nearly twenty-one years in the world
              with very little to distress or vex her.

              She was the youngest of the two daughters of a most affectionate,
              indulgent father; and had, in consequence of her sister's marriage,
              been mistress of his house from a very early period.  Her mother
              had died too long ago for her to have more than an indistinct
              remembrance of her caresses; and her place had been supplied
              by an excellent woman as governess, who had fallen little short
              of a mother in affection.

              Sixteen years had Miss Taylor been in Mr. Woodhouse's family,
              less as a governess than a friend, very fond of both daughters,
              but particularly of Emma.  Between _them_ it was more the intimacy
              of sisters.  Even before Miss Taylor had ceased to hold the nominal
              office of governess, the mildness of her temper had hardly allowed
              her to impose any restraint; and the shadow of authority being
              now long passed away, they had been living together as friend and
              friend very mutually attached, and Emma doing just what she liked;
              highly esteeming Miss Taylor's judgment, but directed chiefly by
              her own.

              The real evils, indeed, of Emma's situation were the power of having
              rather too much her own way, and a disposition to think a little
              too well of herself; these were the disadvantages which threatened
              alloy to her many enjoyments.  The danger, however, was at present
              so unperceived, that they did not by any means rank as misfortunes
              with her.

              Sorrow came--a gentle sorrow--but not at all in the shape of any
              disagreeable consciousness.--Miss Taylor married.  It was Miss
              Taylor's loss which first brought grief.  It was on the wedding-day
              of this beloved friend that Emma first sat in mournful thought
              of any continuance.  The wedding over, and the bride-people gone,
              her father and herself were left to dine together, with no prospect
              of a third to cheer a long evening.  Her father composed himself
              to sleep after dinner, as usual, and she had then only to sit
              and think of what she had lost.

              The event had every promise of happiness for her friend.  Mr. Weston
              was a man of unexceptionable character, easy fortune, suitable age,
              and pleasant manners; and there was some satisfaction in considering
              with what self-denying, generous friendship she had always wished
              and promoted the match; but it was a black morning's work for her.
              The want of Miss Taylor would be felt every hour of every day.
              She recalled her past kindness--the kindness, the affection of sixteen
              years--how she had taught and how she had played with her from five
              years old--how she had devoted all her powers to attach and amuse
              her in health--and how nursed her through the various illnesses
              of childhood.  A large debt of gratitude was owing here; but the
              intercourse of the last seven years, the equal footing and perfect
              unreserve which had soon followed Isabella's marriage, on their
              being left to each other, was yet a dearer, tenderer recollection.
              She had been a friend and companion such as few possessed: intelligent,
              well-informed, useful, gentle, knowing all the ways of the family,
              interested in all its concerns, and peculiarly interested in herself,
              in every pleasure, every scheme of hers--one to whom she could speak
              every thought as it arose, and who had such an affection for her
              as could never find fault.

              How was she to bear the change?--It was true that her friend was
              going only half a mile from them; but Emma was aware that great must
              be the difference between a Mrs. Weston, only half a mile from them,
              and a Miss Taylor in the house; and with all her advantages,
              natural and domestic, she was now in great danger of suffering
              from intellectual solitude.  She dearly loved her father, but he
              was no companion for her.  He could not meet her in conversation,
              rational or playful.

              The evil of the actual disparity in their ages (and Mr. Woodhouse had
              not married early) was much increased by his constitution and habits;
              for having been a valetudinarian all his life, without activity
              of mind or body, he was a much older man in ways than in years;
              and though everywhere beloved for the friendliness of his heart
              and his amiable temper, his talents could not have recommended him
              at any time.

              Her sister, though comparatively but little removed by matrimony,
              being settled in London, only sixteen miles off, was much beyond
              her daily reach; and many a long October and November evening must
              be struggled through at Hartfield, before Christmas brought the next
              visit from Isabella and her husband, and their little children,
              to fill the house, and give her pleasant society again.

              Highbury, the large and populous village, almost amounting to a town,
              to which Hartfield, in spite of its separate lawn, and shrubberies,
              and name, did really belong, afforded her no equals.  The Woodhouses
              were first in consequence there.  All looked up to them.  She had
              many acquaintance in the place, for her father was universally civil,
              but not one among them who could be accepted in lieu of Miss
              Taylor for even half a day.  It was a melancholy change; and Emma
              could not but sigh over it, and wish for impossible things,
              till her father awoke, and made it necessary to be cheerful.
              His spirits required support.  He was a nervous man, easily depressed;
              fond of every body that he was used to, and hating to part with them;
              hating change of every kind.  Matrimony, as the origin of change,
              was always disagreeable; and he was by no means yet reconciled
              to his own daughter's marrying, nor could ever speak of her but
              with compassion, though it had been entirely a match of affection,
              when he was now obliged to part with Miss Taylor too; and from
              his habits of gentle selfishness, and of being never able to
              suppose that other people could feel differently from himself,
              he was very much disposed to think Miss Taylor had done as sad
              a thing for herself as for them, and would have been a great deal
              happier if she had spent all the rest of her life at Hartfield.
              Emma smiled and chatted as cheerfully as she could, to keep him
              from such thoughts; but when tea came, it was impossible for him
              not to say exactly as he had said at dinner,

              "Poor Miss Taylor!--I wish she were here again.  What a pity it
              is that Mr. Weston ever thought of her!"

              "I cannot agree with you, papa; you know I cannot.  Mr. Weston is such
              a good-humoured, pleasant, excellent man, that he thoroughly deserves
              a good wife;--and you would not have had Miss Taylor live with us
              for ever, and bear all my odd humours, when she might have a house of her own?"

              "A house of her own!--But where is the advantage of a house of her own?
              This is three times as large.--And you have never any odd humours,
              my dear."

              "How often we shall be going to see them, and they coming to see
              us!--We shall be always meeting! _We_ must begin; we must go and pay
              wedding visit very soon."

              "My dear, how am I to get so far? Randalls is such a distance.
              I could not walk half so far."

              "No, papa, nobody thought of your walking.  We must go in the carriage,
              to be sure."

              "The carriage! But James will not like to put the horses to for
              such a little way;--and where are the poor horses to be while we
              are paying our visit?"

              "They are to be put into Mr. Weston's stable, papa.  You know we
              have settled all that already.  We talked it all over with Mr. Weston
              last night.  And as for James, you may be very sure he will always like
              going to Randalls, because of his daughter's being housemaid there.
              I only doubt whether he will ever take us anywhere else.  That was
              your doing, papa.  You got Hannah that good place.  Nobody thought
              of Hannah till you mentioned her--James is so obliged to you!"

              "I am very glad I did think of her.  It was very lucky, for I would
              not have had poor James think himself slighted upon any account;
              and I am sure she will make a very good servant: she is a civil,
              pretty-spoken girl; I have a great opinion of her.  Whenever I see her,
              she always curtseys and asks me how I do, in a very pretty manner;
              and when you have had her here to do needlework, I observe she
              always turns the lock of the door the right way and never bangs it.
              I am sure she will be an excellent servant; and it will be a great
              comfort to poor Miss Taylor to have somebody about her that she is
              used to see.  Whenever James goes over to see his daughter, you know,
              she will be hearing of us.  He will be able to tell her how we
              all are."

              Emma spared no exertions to maintain this happier flow of ideas,
              and hoped, by the help of backgammon, to get her father tolerably
              through the evening, and be attacked by no regrets but her own.
              The backgammon-table was placed; but a visitor immediately afterwards
              walked in and made it unnecessary.

              Mr. Knightley, a sensible man about seven or eight-and-thirty, was not
              only a very old and intimate friend of the family, but particularly
              connected with it, as the elder brother of Isabella's husband.
              He lived about a mile from Highbury, was a frequent visitor,
              and always welcome, and at this time more welcome than usual,
              as coming directly from their mutual connexions in London.  He had
              returned to a late dinner, after some days' absence, and now walked
              up to Hartfield to say that all were well in Brunswick Square.
              It was a happy circumstance, and animated Mr. Woodhouse for some time.
              Mr. Knightley had a cheerful manner, which always did him good;
              and his many inquiries after "poor Isabella" and her children were
              answered most satisfactorily.  When this was over, Mr. Woodhouse
              gratefully observed, "It is very kind of you, Mr. Knightley, to come
              out at this late hour to call upon us.  I am afraid you must have
              had a shocking walk."

              "Not at all, sir.  It is a beautiful moonlight night; and so mild
              that I must draw back from your great fire."

              "But you must have found it very damp and dirty.  I wish you may
              not catch cold."

              "Dirty, sir! Look at my shoes.  Not a speck on them."

              "Well! that is quite surprising, for we have had a vast deal
              of rain here.  It rained dreadfully hard for half an hour
              while we were at breakfast.  I wanted them to put off the wedding."

              "By the bye--I have not wished you joy.  Being pretty well aware
              of what sort of joy you must both be feeling, I have been in no hurry
              with my congratulations; but I hope it all went off tolerably well.
              How did you all behave? Who cried most?"

              "Ah! poor Miss Taylor! 'Tis a sad business."

              "Poor Mr. and Miss Woodhouse, if you please; but I cannot possibly
              say `poor Miss Taylor.' I have a great regard for you and Emma;
              but when it comes to the question of dependence or independence!--At
              any rate, it must be better to have only one to please than two."

              "Especially when _one_ of those two is such a fanciful, troublesome creature!"
              said Emma playfully.  "That is what you have in your head,
              I know--and what you would certainly say if my father were not by."

              "I believe it is very true, my dear, indeed," said Mr. Woodhouse,
              with a sigh.  "I am afraid I am sometimes very fanciful and troublesome."

              "My dearest papa! You do not think I could mean _you_, or suppose
              Mr. Knightley to mean _you_.  What a horrible idea! Oh no! I meant
              only myself.  Mr. Knightley loves to find fault with me, you know--
              in a joke--it is all a joke.  We always say what we like to one another."

              Mr. Knightley, in fact, was one of the few people who could see
              faults in Emma Woodhouse, and the only one who ever told her of them:
              and though this was not particularly agreeable to Emma herself,
              she knew it would be so much less so to her father, that she would
              not have him really suspect such a circumstance as her not being
              thought perfect by every body.

              "Emma knows I never flatter her," said Mr. Knightley, "but I
              meant no reflection on any body.  Miss Taylor has been used
              to have two persons to please; she will now have but one.
              The chances are that she must be a gainer."

              "Well," said Emma, willing to let it pass--"you want to hear
              about the wedding; and I shall be happy to tell you, for we all
              behaved charmingly.  Every body was punctual, every body in their
              best looks: not a tear, and hardly a long face to be seen.  Oh no;
              we all felt that we were going to be only half a mile apart,
              and were sure of meeting every day."

              "Dear Emma bears every thing so well," said her father.
              "But, Mr. Knightley, she is really very sorry to lose poor Miss Taylor,
              and I am sure she _will_ miss her more than she thinks for."

              Emma turned away her head, divided between tears and smiles.
              "It is impossible that Emma should not miss such a companion,"
              said Mr. Knightley.  "We should not like her so well as we do, sir,
              if we could suppose it; but she knows how much the marriage is to
              Miss Taylor's advantage; she knows how very acceptable it must be,
              at Miss Taylor's time of life, to be settled in a home of her own,
              and how important to her to be secure of a comfortable provision,
              and therefore cannot allow herself to feel so much pain as pleasure.
              Every friend of Miss Taylor must be glad to have her so happily
              married."

              "And you have forgotten one matter of joy to me," said Emma,
              "and a very considerable one--that I made the match myself.
              I made the match, you know, four years ago; and to have it take place,
              and be proved in the right, when so many people said Mr. Weston would
              never marry again, may comfort me for any thing."

              Mr. Knightley shook his head at her.  Her father fondly replied,
              "Ah! my dear, I wish you would not make matches and foretell things,
              for whatever you say always comes to pass.  Pray do not make any
              more matches."

              "I promise you to make none for myself, papa; but I must, indeed,
              for other people.  It is the greatest amusement in the world! And
              after such success, you know!--Every body said that Mr. Weston would
              never marry again.  Oh dear, no! Mr. Weston, who had been a widower
              so long, and who seemed so perfectly comfortable without a wife,
              so constantly occupied either in his business in town or among his
              friends here, always acceptable wherever he went, always cheerful--
              Mr. Weston need not spend a single evening in the year alone if he did
              not like it.  Oh no! Mr. Weston certainly would never marry again.
              Some people even talked of a promise to his wife on her deathbed,
              and others of the son and the uncle not letting him.  All manner
              of solemn nonsense was talked on the subject, but I believed none
              of it.

              "Ever since the day--about four years ago--that Miss Taylor and I
              met with him in Broadway Lane, when, because it began to drizzle,
              he darted away with so much gallantry, and borrowed two umbrellas
              for us from Farmer Mitchell's, I made up my mind on the subject.
              I planned the match from that hour; and when such success has blessed
              me in this instance, dear papa, you cannot think that I shall leave
              off match-making."

              "I do not understand what you mean by `success,'" said Mr. Knightley.
              "Success supposes endeavour.  Your time has been properly and
              delicately spent, if you have been endeavouring for the last four
              years to bring about this marriage.  A worthy employment for a young
              lady's mind! But if, which I rather imagine, your making the match,
              as you call it, means only your planning it, your saying to yourself
              one idle day, `I think it would be a very good thing for Miss Taylor
              if Mr. Weston were to marry her,' and saying it again to yourself
              every now and then afterwards, why do you talk of success? Where
              is your merit? What are you proud of? You made a lucky guess;
              and _that_ is all that can be said."

              "And have you never known the pleasure and triumph of a lucky guess?--
              I pity you.--I thought you cleverer--for, depend upon it a lucky
              guess is never merely luck.  There is always some talent in it.
              And as to my poor word `success,' which you quarrel with, I do not
              know that I am so entirely without any claim to it.  You have drawn
              two pretty pictures; but I think there may be a third--a something
              between the do-nothing and the do-all. If I had not promoted Mr. Weston's
              visits here, and given many little encouragements, and smoothed
              many little matters, it might not have come to any thing after all.
              I think you must know Hartfield enough to comprehend that."

              "A straightforward, open-hearted man like Weston, and a rational,
              unaffected woman like Miss Taylor, may be safely left to manage their
              own concerns.  You are more likely to have done harm to yourself,
              than good to them, by interference."

              "Emma never thinks of herself, if she can do good to others,"
              rejoined Mr. Woodhouse, understanding but in part.  "But, my dear,
              pray do not make any more matches; they are silly things, and break up
              one's family circle grievously."

              "Only one more, papa; only for Mr. Elton.  Poor Mr. Elton! You
              like Mr. Elton, papa,--I must look about for a wife for him.
              There is nobody in Highbury who deserves him--and he has been
              here a whole year, and has fitted up his house so comfortably,
              that it would be a shame to have him single any longer--and I thought
              when he was joining their hands to-day, he looked so very much as if
              he would like to have the same kind office done for him! I think
              very well of Mr. Elton, and this is the only way I have of doing
              him a service."

              "Mr. Elton is a very pretty young man, to be sure, and a very
              good young man, and I have a great regard for him.  But if you
              want to shew him any attention, my dear, ask him to come
              and dine with us some day.  That will be a much better thing.
              I dare say Mr. Knightley will be so kind as to meet him."

              "With a great deal of pleasure, sir, at any time," said Mr. Knightley,
              laughing, "and I agree with you entirely, that it will be a much
              better thing.  Invite him to dinner, Emma, and help him to the best
              of the fish and the chicken, but leave him to chuse his own wife.
              Depend upon it, a man of six or seven-and-twenty can take care
              of himself."



              CHAPTER II


              Mr. Weston was a native of Highbury, and born of a respectable family,
              which for the last two or three generations had been rising into
              gentility and property.  He had received a good education, but,
              on succeeding early in life to a small independence, had become
              indisposed for any of the more homely pursuits in which his brothers
              were engaged, and had satisfied an active, cheerful mind and social
              temper by entering into the militia of his county, then embodied.

              Captain Weston was a general favourite; and when the chances
              of his military life had introduced him to Miss Churchill,
              of a great Yorkshire family, and Miss Churchill fell in love
              with him, nobody was surprized, except her brother and his wife,
              who had never seen him, and who were full of pride and importance,
              which the connexion would offend.

              Miss Churchill, however, being of age, and with the full command
              of her fortune--though her fortune bore no proportion to the
              family-estate--was not to be dissuaded from the marriage, and it
              took place, to the infinite mortification of Mr. and Mrs. Churchill,
              who threw her off with due decorum.  It was an unsuitable connexion,
              and did not produce much happiness.  Mrs. Weston ought to have found
              more in it, for she had a husband whose warm heart and sweet temper
              made him think every thing due to her in return for the great goodness
              of being in love with him; but though she had one sort of spirit,
              she had not the best.  She had resolution enough to pursue
              her own will in spite of her brother, but not enough to refrain
              from unreasonable regrets at that brother's unreasonable anger,
              nor from missing the luxuries of her former home.  They lived beyond
              their income, but still it was nothing in comparison of Enscombe:
              she did not cease to love her husband, but she wanted at once
              to be the wife of Captain Weston, and Miss Churchill of Enscombe.

              Captain Weston, who had been considered, especially by the Churchills,
              as making such an amazing match, was proved to have much the worst
              of the bargain; for when his wife died, after a three years' marriage,
              he was rather a poorer man than at first, and with a child to maintain.
              From the expense of the child, however, he was soon relieved.
              The boy had, with the additional softening claim of a lingering
              illness of his mother's, been the means of a sort of reconciliation;
              and Mr. and Mrs. Churchill, having no children of their own,
              nor any other young creature of equal kindred to care for, offered to
              take the whole charge of the little Frank soon after her decease.
              Some scruples and some reluctance the widower-father may be supposed
              to have felt; but as they were overcome by other considerations,
              the child was given up to the care and the wealth of the Churchills,
              and he had only his own comfort to seek, and his own situation to
              improve as he could.

              A complete change of life became desirable.  He quitted the militia
              and engaged in trade, having brothers already established in a
              good way in London, which afforded him a favourable opening.
              It was a concern which brought just employment enough.  He had still
              a small house in Highbury, where most of his leisure days were spent;
              and between useful occupation and the pleasures of society,
              the next eighteen or twenty years of his life passed cheerfully away.
              He had, by that time, realised an easy competence--enough to secure
              the purchase of a little estate adjoining Highbury, which he had
              always longed for--enough to marry a woman as portionless even
              as Miss Taylor, and to live according to the wishes of his own
              friendly and social disposition.

              It was now some time since Miss Taylor had begun to influence
              his schemes; but as it was not the tyrannic influence of youth
              on youth, it had not shaken his determination of never settling
              till he could purchase Randalls, and the sale of Randalls was long
              looked forward to; but he had gone steadily on, with these objects
              in view, till they were accomplished.  He had made his fortune,
              bought his house, and obtained his wife; and was beginning a new
              period of existence, with every probability of greater happiness
              than in any yet passed through.  He had never been an unhappy man;
              his own temper had secured him from that, even in his first marriage;
              but his second must shew him how delightful a well-judging and truly
              amiable woman could be, and must give him the pleasantest proof
              of its being a great deal better to choose than to be chosen,
              to excite gratitude than to feel it.

              He had only himself to please in his choice: his fortune was
              his own; for as to Frank, it was more than being tacitly brought
              up as his uncle's heir, it had become so avowed an adoption
              as to have him assume the name of Churchill on coming of age.
              It was most unlikely, therefore, that he should ever want his
              father's assistance.  His father had no apprehension of it.
              The aunt was a capricious woman, and governed her husband entirely;
              but it was not in Mr. Weston's nature to imagine that any caprice
              could be strong enough to affect one so dear, and, as he believed,
              so deservedly dear.  He saw his son every year in London,
              and was proud of him; and his fond report of him as a very fine
              young man had made Highbury feel a sort of pride in him too.
              He was looked on as sufficiently belonging to the place to make his
              merits and prospects a kind of common concern.

              Mr. Frank Churchill was one of the boasts of Highbury, and a lively
              curiosity to see him prevailed, though the compliment was so little
              returned that he had never been there in his life.  His coming
              to visit his father had been often talked of but never achieved.

              Now, upon his father's marriage, it was very generally proposed,
              as a most proper attention, that the visit should take place.
              There was not a dissentient voice on the subject, either when
              Mrs. Perry drank tea with Mrs. and Miss Bates, or when Mrs. and
              Miss Bates returned the visit.  Now was the time for Mr. Frank
              Churchill to come among them; and the hope strengthened when it was
              understood that he had written to his new mother on the occasion.
              For a few days, every morning visit in Highbury included some mention
              of the handsome letter Mrs. Weston had received.  "I suppose you
              have heard of the handsome letter Mr. Frank Churchill has written
              to Mrs. Weston? I understand it was a very handsome letter, indeed.
              Mr. Woodhouse told me of it.  Mr. Woodhouse saw the letter, and he
              says he never saw such a handsome letter in his life."

              It was, indeed, a highly prized letter.  Mrs. Weston had, of course,
              formed a very favourable idea of the young man; and such a pleasing
              attention was an irresistible proof of his great good sense,
              and a most welcome addition to every source and every expression
              of congratulation which her marriage had already secured.  She felt
              herself a most fortunate woman; and she had lived long enough
              to know how fortunate she might well be thought, where the only
              regret was for a partial separation from friends whose friendship
              for her had never cooled, and who could ill bear to part with her.

              She knew that at times she must be missed; and could not think,
              without pain, of Emma's losing a single pleasure, or suffering
              an hour's ennui, from the want of her companionableness: but dear
              Emma was of no feeble character; she was more equal to her situation
              than most girls would have been, and had sense, and energy,
              and spirits that might be hoped would bear her well and happily
              through its little difficulties and privations.  And then there was
              such comfort in the very easy distance of Randalls from Hartfield,
              so convenient for even solitary female walking, and in Mr. Weston's
              disposition and circumstances, which would make the approaching
              season no hindrance to their spending half the evenings in the
              week together.

              Her situation was altogether the subject of hours of gratitude
              to Mrs. Weston, and of moments only of regret; and her
              satisfaction--her more than satisfaction--her cheerful enjoyment,
              was so just and so apparent, that Emma, well as she knew her father,
              was sometimes taken by surprize at his being still able to pity
              `poor Miss Taylor,' when they left her at Randalls in the centre
              of every domestic comfort, or saw her go away in the evening
              attended by her pleasant husband to a carriage of her own.
              But never did she go without Mr. Woodhouse's giving a gentle sigh,
              and saying, "Ah, poor Miss Taylor! She would be very glad to stay."

              There was no recovering Miss Taylor--nor much likelihood of
              ceasing to pity her; but a few weeks brought some alleviation
              to Mr. Woodhouse.  The compliments of his neighbours were over;
              he was no longer teased by being wished joy of so sorrowful an event;
              and the wedding-cake, which had been a great distress to him,
              was all eat up.  His own stomach could bear nothing rich, and he
              could never believe other people to be different from himself.
              What was unwholesome to him he regarded as unfit for any body;
              and he had, therefore, earnestly tried to dissuade them from having
              any wedding-cake at all, and when that proved vain, as earnestly
              tried to prevent any body's eating it.  He had been at the pains
              of consulting Mr. Perry, the apothecary, on the subject.  Mr. Perry
              was an intelligent, gentlemanlike man, whose frequent visits were one
              of the comforts of Mr. Woodhouse's life; and upon being applied to,
              he could not but acknowledge (though it seemed rather against the
              bias of inclination) that wedding-cake might certainly disagree
              with many--perhaps with most people, unless taken moderately.
              With such an opinion, in confirmation of his own, Mr. Woodhouse hoped
              to influence every visitor of the newly married pair; but still the
              cake was eaten; and there was no rest for his benevolent nerves till
              it was all gone.

              There was a strange rumour in Highbury of all the little Perrys
              being seen with a slice of Mrs. Weston's wedding-cake in their
              hands: but Mr. Woodhouse would never believe it.



              CHAPTER III


              Mr. Woodhouse was fond of society in his own way.  He liked very much
              to have his friends come and see him; and from various united causes,
              from his long residence at Hartfield, and his good nature,
              from his fortune, his house, and his daughter, he could command the
              visits of his own little circle, in a great measure, as he liked.
              He had not much intercourse with any families beyond that circle;
              his horror of late hours, and large dinner-parties, made him unfit
              for any acquaintance but such as would visit him on his own terms.
              Fortunately for him, Highbury, including Randalls in the same parish,
              and Donwell Abbey in the parish adjoining, the seat of Mr. Knightley,
              comprehended many such.  Not unfrequently, through Emma's persuasion,
              he had some of the chosen and the best to dine with him: but evening
              parties were what he preferred; and, unless he fancied himself at any
              time unequal to company, there was scarcely an evening in the week
              in which Emma could not make up a card-table for him.

              Real, long-standing regard brought the Westons and Mr. Knightley;
              and by Mr. Elton, a young man living alone without liking it,
              the privilege of exchanging any vacant evening of his own blank solitude
              for the elegancies and society of Mr. Woodhouse's drawing-room,
              and the smiles of his lovely daughter, was in no danger of being
              thrown away.

              After these came a second set; among the most come-at-able
              of whom were Mrs. and Miss Bates, and Mrs. Goddard, three ladies
              almost always at the service of an invitation from Hartfield,
              and who were fetched and carried home so often, that Mr. Woodhouse
              thought it no hardship for either James or the horses.  Had it
              taken place only once a year, it would have been a grievance.

              Mrs. Bates, the widow of a former vicar of Highbury, was a
              very old lady, almost past every thing but tea and quadrille.
              She lived with her single daughter in a very small way, and was
              considered with all the regard and respect which a harmless old lady,
              under such untoward circumstances, can excite.  Her daughter enjoyed
              a most uncommon degree of popularity for a woman neither young,
              handsome, rich, nor married.  Miss Bates stood in the very worst
              predicament in the world for having much of the public favour;
              and she had no intellectual superiority to make atonement to herself,
              or frighten those who might hate her into outward respect.
              She had never boasted either beauty or cleverness.  Her youth
              had passed without distinction, and her middle of life was devoted
              to the care of a failing mother, and the endeavour to make a small
              income go as far as possible.  And yet she was a happy woman,
              and a woman whom no one named without good-will.  It was her own
              universal good-will and contented temper which worked such wonders.
              She loved every body, was interested in every body's happiness,
              quicksighted to every body's merits; thought herself a most fortunate
              creature, and surrounded with blessings in such an excellent mother,
              and so many good neighbours and friends, and a home that wanted
              for nothing.  The simplicity and cheerfulness of her nature,
              her contented and grateful spirit, were a recommendation to every body,
              and a mine of felicity to herself.  She was a great talker upon
              little matters, which exactly suited Mr. Woodhouse, full of trivial
              communications and harmless gossip.

              Mrs. Goddard was the mistress of a School--not of a seminary,
              or an establishment, or any thing which professed, in long sentences of
              refined nonsense, to combine liberal acquirements with elegant morality,
              upon new principles and new systems--and where young ladies for
              enormous pay might be screwed out of health and into vanity--but
              a real, honest, old-fashioned Boarding-school, where a reasonable
              quantity of accomplishments were sold at a reasonable price,
              and where girls might be sent to be out of the way, and scramble
              themselves into a little education, without any danger of coming
              back prodigies.  Mrs. Goddard's school was in high repute--and
              very deservedly; for Highbury was reckoned a particularly healthy
              spot: she had an ample house and garden, gave the children plenty
              of wholesome food, let them run about a great deal in the summer,
              and in winter dressed their chilblains with her own hands.
              It was no wonder that a train of twenty young couple now walked
              after her to church.  She was a plain, motherly kind of woman,
              who had worked hard in her youth, and now thought herself entitled
              to the occasional holiday of a tea-visit; and having formerly
              owed much to Mr. Woodhouse's kindness, felt his particular claim
              on her to leave her neat parlour, hung round with fancy-work,
              whenever she could, and win or lose a few sixpences by his fireside.

              These were the ladies whom Emma found herself very frequently
              able to collect; and happy was she, for her father's sake,
              in the power; though, as far as she was herself concerned,
              it was no remedy for the absence of Mrs. Weston.  She was delighted
              to see her father look comfortable, and very much pleased with
              herself for contriving things so well; but the quiet prosings
              of three such women made her feel that every evening so spent
              was indeed one of the long evenings she had fearfully anticipated.

              As she sat one morning, looking forward to exactly such a close
              of the present day, a note was brought from Mrs. Goddard, requesting,
              in most respectful terms, to be allowed to bring Miss Smith with her;
              a most welcome request: for Miss Smith was a girl of seventeen,
              whom Emma knew very well by sight, and had long felt an interest in,
              on account of her beauty.  A very gracious invitation was returned,
              and the evening no longer dreaded by the fair mistress of the mansion.

              Harriet Smith was the natural daughter of somebody.  Somebody had
              placed her, several years back, at Mrs. Goddard's school,
              and somebody had lately raised her from the condition of scholar
              to that of parlour-boarder. This was all that was generally known
              of her history.  She had no visible friends but what had been
              acquired at Highbury, and was now just returned from a long visit
              in the country to some young ladies who had been at school there with her.

              She was a very pretty girl, and her beauty happened to be of a sort
              which Emma particularly admired.  She was short, plump, and fair,
              with a fine bloom, blue eyes, light hair, regular features,
              and a look of great sweetness, and, before the end of the evening,
              Emma was as much pleased with her manners as her person, and quite
              determined to continue the acquaintance.

              She was not struck by any thing remarkably clever in Miss Smith's
              conversation, but she found her altogether very engaging--not
              inconveniently shy, not unwilling to talk--and yet so far from pushing,
              shewing so proper and becoming a deference, seeming so pleasantly
              grateful for being admitted to Hartfield, and so artlessly
              impressed by the appearance of every thing in so superior a style
              to what she had been used to, that she must have good sense,
              and deserve encouragement.  Encouragement should be given.
              Those soft blue eyes, and all those natural graces, should not be
              wasted on the inferior society of Highbury and its connexions.
              The acquaintance she had already formed were unworthy of her.
              The friends from whom she had just parted, though very good sort
              of people, must be doing her harm.  They were a family of the name
              of Martin, whom Emma well knew by character, as renting a large farm
              of Mr. Knightley, and residing in the parish of Donwell--very creditably,
              she believed--she knew Mr. Knightley thought highly of them--but they
              must be coarse and unpolished, and very unfit to be the intimates
              of a girl who wanted only a little more knowledge and elegance
              to be quite perfect.  _She_ would notice her; she would improve her;
              she would detach her from her bad acquaintance, and introduce her
              into good society; she would form her opinions and her manners.
              It would be an interesting, and certainly a very kind undertaking;
              highly becoming her own situation in life, her leisure, and powers.

              She was so busy in admiring those soft blue eyes, in talking
              and listening, and forming all these schemes in the in-betweens, that
              the evening flew away at a very unusual rate; and the supper-table,
              which always closed such parties, and for which she had been
              used to sit and watch the due time, was all set out and ready,
              and moved forwards to the fire, before she was aware.  With an
              alacrity beyond the common impulse of a spirit which yet was never
              indifferent to the credit of doing every thing well and attentively,
              with the real good-will of a mind delighted with its own ideas,
              did she then do all the honours of the meal, and help and recommend
              the minced chicken and scalloped oysters, with an urgency which she
              knew would be acceptable to the early hours and civil scruples of their guests.

              Upon such occasions poor Mr. Woodhouses feelings were in sad warfare.
              He loved to have the cloth laid, because it had been the fashion
              of his youth, but his conviction of suppers being very unwholesome
              made him rather sorry to see any thing put on it; and while his
              hospitality would have welcomed his visitors to every thing,
              his care for their health made him grieve that they would eat.

              Such another small basin of thin gruel as his own was all that
              he could, with thorough self-approbation, recommend; though he
              might constrain himself, while the ladies were comfortably clearing
              the nicer things, to say:

              "Mrs. Bates, let me propose your venturing on one of these eggs.
              An egg boiled very soft is not unwholesome.  Serle understands boiling
              an egg better than any body.  I would not recommend an egg boiled
              by any body else; but you need not be afraid, they are very small,
              you see--one of our small eggs will not hurt you.  Miss Bates,
              let Emma help you to a _little_ bit of tart--a _very_ little bit.
              Ours are all apple-tarts. You need not be afraid of unwholesome
              preserves here.  I do not advise the custard.  Mrs. Goddard, what say
              you to _half_ a glass of wine? A _small_ half-glass, put into a tumbler
              of water? I do not think it could disagree with you."

              Emma allowed her father to talk--but supplied her visitors in
              a much more satisfactory style, and on the present evening had
              particular pleasure in sending them away happy.  The happiness
              of Miss Smith was quite equal to her intentions.  Miss Woodhouse
              was so great a personage in Highbury, that the prospect of the
              introduction had given as much panic as pleasure; but the humble,
              grateful little girl went off with highly gratified feelings,
              delighted with the affability with which Miss Woodhouse had treated
              her all the evening, and actually shaken hands with her at last!



              CHAPTER IV


              Harriet Smith's intimacy at Hartfield was soon a settled thing.
              Quick and decided in her ways, Emma lost no time in inviting, encouraging,
              and telling her to come very often; and as their acquaintance increased,
              so did their satisfaction in each other.  As a walking companion,
              Emma had very early foreseen how useful she might find her.
              In that respect Mrs. Weston's loss had been important.  Her father
              never went beyond the shrubbery, where two divisions of the ground
              sufficed him for his long walk, or his short, as the year varied;
              and since Mrs. Weston's marriage her exercise had been too much confined.
              She had ventured once alone to Randalls, but it was not pleasant;
              and a Harriet Smith, therefore, one whom she could summon at any
              time to a walk, would be a valuable addition to her privileges.
              But in every respect, as she saw more of her, she approved her,
              and was confirmed in all her kind designs.

              Harriet certainly was not clever, but she had a sweet, docile,
              grateful disposition, was totally free from conceit, and only desiring
              to be guided by any one she looked up to.  Her early attachment
              to herself was very amiable; and her inclination for good company,
              and power of appreciating what was elegant and clever, shewed that
              there was no want of taste, though strength of understanding must
              not be expected.  Altogether she was quite convinced of Harriet
              Smith's being exactly the young friend she wanted--exactly the
              something which her home required.  Such a friend as Mrs. Weston
              was out of the question.  Two such could never be granted.
              Two such she did not want.  It was quite a different sort of thing,
              a sentiment distinct and independent.  Mrs. Weston was the object
              of a regard which had its basis in gratitude and esteem.
              Harriet would be loved as one to whom she could be useful.
              For Mrs. Weston there was nothing to be done; for Harriet every thing.

              Her first attempts at usefulness were in an endeavour to find out who
              were the parents, but Harriet could not tell.  She was ready to tell
              every thing in her power, but on this subject questions were vain.
              Emma was obliged to fancy what she liked--but she could never
              believe that in the same situation _she_ should not have discovered
              the truth.  Harriet had no penetration.  She had been satisfied
              to hear and believe just what Mrs. Goddard chose to tell her;
              and looked no farther.

              Mrs. Goddard, and the teachers, and the girls and the affairs of the
              school in general, formed naturally a great part of the conversation--and
              but for her acquaintance with the Martins of Abbey-Mill Farm,
              it must have been the whole.  But the Martins occupied her thoughts
              a good deal; she had spent two very happy months with them,
              and now loved to talk of the pleasures of her visit, and describe
              the many comforts and wonders of the place.  Emma encouraged her
              talkativeness--amused by such a picture of another set of beings,
              and enjoying the youthful simplicity which could speak with so much
              exultation of Mrs. Martin's having "_two_ parlours, two very good parlours,
              indeed; one of them quite as large as Mrs. Goddard's drawing-room;
              and of her having an upper maid who had lived five-and-twenty years
              with her; and of their having eight cows, two of them Alderneys,
              and one a little Welch cow, a very pretty little Welch cow indeed;
              and of Mrs. Martin's saying as she was so fond of it, it should be
              called _her_ cow; and of their having a very handsome summer-house
              in their garden, where some day next year they were all to drink
              tea:--a very handsome summer-house, large enough to hold a dozen people."

              For some time she was amused, without thinking beyond the immediate cause;
              but as she came to understand the family better, other feelings arose.
              She had taken up a wrong idea, fancying it was a mother and daughter,
              a son and son's wife, who all lived together; but when it appeared
              that the Mr. Martin, who bore a part in the narrative, and was always
              mentioned with approbation for his great good-nature in doing something
              or other, was a single man; that there was no young Mrs. Martin,
              no wife in the case; she did suspect danger to her poor little
              friend from all this hospitality and kindness, and that, if she
              were not taken care of, she might be required to sink herself forever.

              With this inspiriting notion, her questions increased in number
              and meaning; and she particularly led Harriet to talk more of Mr. Martin,
              and there was evidently no dislike to it.  Harriet was very ready
              to speak of the share he had had in their moonlight walks and merry
              evening games; and dwelt a good deal upon his being so very good-humoured
              and obliging.  He had gone three miles round one day in order to bring
              her some walnuts, because she had said how fond she was of them,
              and in every thing else he was so very obliging.  He had his
              shepherd's son into the parlour one night on purpose to sing to her.
              She was very fond of singing.  He could sing a little himself.
              She believed he was very clever, and understood every thing.
              He had a very fine flock, and, while she was with them,
              he had been bid more for his wool than any body in the country.
              She believed every body spoke well of him.  His mother and sisters
              were very fond of him.  Mrs. Martin had told her one day (and there
              was a blush as she said it,) that it was impossible for any body
              to be a better son, and therefore she was sure, whenever he married,
              he would make a good husband.  Not that she _wanted_ him to marry.
              She was in no hurry at all.

              "Well done, Mrs. Martin!" thought Emma.  "You know what you are about."

              "And when she had come away, Mrs. Martin was so very kind as to send
              Mrs. Goddard a beautiful goose--the finest goose Mrs. Goddard had
              ever seen.  Mrs. Goddard had dressed it on a Sunday, and asked all
              the three teachers, Miss Nash, and Miss Prince, and Miss Richardson,
              to sup with her."

              "Mr. Martin, I suppose, is not a man of information beyond the line
              of his own business? He does not read?"

              "Oh yes!--that is, no--I do not know--but I believe he has
              read a good deal--but not what you would think any thing of.
              He reads the Agricultural Reports, and some other books that lay
              in one of the window seats--but he reads all _them_ to himself.
              But sometimes of an evening, before we went to cards, he would read
              something aloud out of the Elegant Extracts, very entertaining.
              And I know he has read the Vicar of Wakefield.  He never read the
              Romance of the Forest, nor The Children of the Abbey.  He had never
              heard of such books before I mentioned them, but he is determined
              to get them now as soon as ever he can."

              The next question was--

              "What sort of looking man is Mr. Martin?"

              "Oh! not handsome--not at all handsome.  I thought him very plain
              at first, but I do not think him so plain now.  One does not, you know,
              after a time.  But did you never see him? He is in Highbury every
              now and then, and he is sure to ride through every week in his way
              to Kingston.  He has passed you very often."

              "That may be, and I may have seen him fifty times, but without
              having any idea of his name.  A young farmer, whether on horseback
              or on foot, is the very last sort of person to raise my curiosity.
              The yeomanry are precisely the order of people with whom I feel I
              can have nothing to do.  A degree or two lower, and a creditable
              appearance might interest me; I might hope to be useful to their
              families in some way or other.  But a farmer can need none of my help,
              and is, therefore, in one sense, as much above my notice as in every
              other he is below it."

              "To be sure.  Oh yes! It is not likely you should ever have
              observed him; but he knows you very well indeed--I mean by sight."

              "I have no doubt of his being a very respectable young man.
              I know, indeed, that he is so, and, as such, wish him well.
              What do you imagine his age to be?"

              "He was four-and-twenty the 8th of last June, and my birthday is
              the 23rd just a fortnight and a day's difference--which is very odd."

              "Only four-and-twenty. That is too young to settle.  His mother is
              perfectly right not to be in a hurry.  They seem very comfortable
              as they are, and if she were to take any pains to marry him,
              she would probably repent it.  Six years hence, if he could meet
              with a good sort of young woman in the same rank as his own,
              with a little money, it might be very desirable."

              "Six years hence! Dear Miss Woodhouse, he would be thirty years old!"

              "Well, and that is as early as most men can afford to marry,
              who are not born to an independence.  Mr. Martin, I imagine,
              has his fortune entirely to make--cannot be at all beforehand with
              the world.  Whatever money he might come into when his father died,
              whatever his share of the family property, it is, I dare say,
              all afloat, all employed in his stock, and so forth; and though,
              with diligence and good luck, he may be rich in time, it is next to
              impossible that he should have realised any thing yet."

              "To be sure, so it is.  But they live very comfortably.
              They have no indoors man, else they do not want for any thing;
              and Mrs. Martin talks of taking a boy another year."

              "I wish you may not get into a scrape, Harriet, whenever he does
              marry;--I mean, as to being acquainted with his wife--for though
              his sisters, from a superior education, are not to be altogether
              objected to, it does not follow that he might marry any body at all fit
              for you to notice.  The misfortune of your birth ought to make you
              particularly careful as to your associates.  There can be no doubt
              of your being a gentleman's daughter, and you must support your
              claim to that station by every thing within your own power, or there
              will be plenty of people who would take pleasure in degrading you."

              "Yes, to be sure, I suppose there are.  But while I visit
              at Hartfield, and you are so kind to me, Miss Woodhouse,
              I am not afraid of what any body can do."

              "You understand the force of influence pretty well, Harriet; but I
              would have you so firmly established in good society, as to be
              independent even of Hartfield and Miss Woodhouse.  I want to see you
              permanently well connected, and to that end it will be advisable
              to have as few odd acquaintance as may be; and, therefore, I say
              that if you should still be in this country when Mr. Martin marries,
              I wish you may not be drawn in by your intimacy with the sisters,
              to be acquainted with the wife, who will probably be some mere
              farmer's daughter, without education."

              "To be sure.  Yes.  Not that I think Mr. Martin would ever marry any body
              but what had had some education--and been very well brought up.
              However, I do not mean to set up my opinion against your's--and I
              am sure I shall not wish for the acquaintance of his wife.  I shall
              always have a great regard for the Miss Martins, especially Elizabeth,
              and should be very sorry to give them up, for they are quite as well
              educated as me.  But if he marries a very ignorant, vulgar woman,
              certainly I had better not visit her, if I can help it."

              Emma watched her through the fluctuations of this speech,
              and saw no alarming symptoms of love.  The young man had been
              the first admirer, but she trusted there was no other hold,
              and that there would be no serious difficulty, on Harriet's side,
              to oppose any friendly arrangement of her own.

              They met Mr. Martin the very next day, as they were walking on the
              Donwell road.  He was on foot, and after looking very respectfully
              at her, looked with most unfeigned satisfaction at her companion.
              Emma was not sorry to have such an opportunity of survey;
              and walking a few yards forward, while they talked together, soon made
              her quick eye sufficiently acquainted with Mr. Robert Martin.
              His appearance was very neat, and he looked like a sensible young man,
              but his person had no other advantage; and when he came to be
              contrasted with gentlemen, she thought he must lose all the ground
              he had gained in Harriet's inclination.  Harriet was not insensible
              of manner; she had voluntarily noticed her father's gentleness
              with admiration as well as wonder.  Mr. Martin looked as if he
              did not know what manner was.

              They remained but a few minutes together, as Miss Woodhouse must
              not be kept waiting; and Harriet then came running to her with a
              smiling face, and in a flutter of spirits, which Miss Woodhouse
              hoped very soon to compose.

              "Only think of our happening to meet him!--How very odd! It was
              quite a chance, he said, that he had not gone round by Randalls.
              He did not think we ever walked this road.  He thought we walked
              towards Randalls most days.  He has not been able to get the
              Romance of the Forest yet.  He was so busy the last time he was
              at Kingston that he quite forgot it, but he goes again to-morrow.
              So very odd we should happen to meet! Well, Miss Woodhouse, is he
              like what you expected? What do you think of him? Do you think him
              so very plain?"

              "He is very plain, undoubtedly--remarkably plain:--but that is
              nothing compared with his entire want of gentility.  I had no
              right to expect much, and I did not expect much; but I had no
              idea that he could be so very clownish, so totally without air.
              I had imagined him, I confess, a degree or two nearer gentility."

              "To be sure," said Harriet, in a mortified voice, "he is not
              so genteel as real gentlemen."

              "I think, Harriet, since your acquaintance with us, you have been
              repeatedly in the company of some such very real gentlemen,
              that you must yourself be struck with the difference in Mr. Martin.
              At Hartfield, you have had very good specimens of well educated,
              well bred men.  I should be surprized if, after seeing them,
              you could be in company with Mr. Martin again without perceiving
              him to be a very inferior creature--and rather wondering at
              yourself for having ever thought him at all agreeable before.
              Do not you begin to feel that now? Were not you struck? I am sure
              you must have been struck by his awkward look and abrupt manner,
              and the uncouthness of a voice which I heard to be wholly unmodulated
              as I stood here."

              "Certainly, he is not like Mr. Knightley.  He has not such a fine
              air and way of walking as Mr. Knightley.  I see the difference
              plain enough.  But Mr. Knightley is so very fine a man!"

              "Mr. Knightley's air is so remarkably good that it is not fair
              to compare Mr. Martin with _him_.  You might not see one in a hundred
              with _gentleman_ so plainly written as in Mr. Knightley.  But he is
              not the only gentleman you have been lately used to.  What say you
              to Mr. Weston and Mr. Elton? Compare Mr. Martin with either of _them_.
              Compare their manner of carrying themselves; of walking; of speaking;
              of being silent.  You must see the difference."

              "Oh yes!--there is a great difference.  But Mr. Weston is almost
              an old man.  Mr. Weston must be between forty and fifty."

              "Which makes his good manners the more valuable.  The older a
              person grows, Harriet, the more important it is that their manners
              should not be bad; the more glaring and disgusting any loudness,
              or coarseness, or awkwardness becomes.  What is passable in youth
              is detestable in later age.  Mr. Martin is now awkward and abrupt;
              what will he be at Mr. Weston's time of life?"

              "There is no saying, indeed," replied Harriet rather solemnly.

              "But there may be pretty good guessing.  He will be a completely gross,
              vulgar farmer, totally inattentive to appearances, and thinking
              of nothing but profit and loss."

              "Will he, indeed? That will be very bad."

              "How much his business engrosses him already is very plain from the
              circumstance of his forgetting to inquire for the book you recommended.
              He was a great deal too full of the market to think of any thing
              else--which is just as it should be, for a thriving man.  What has
              he to do with books? And I have no doubt that he _will_ thrive,
              and be a very rich man in time--and his being illiterate and coarse
              need not disturb _us_."

              "I wonder he did not remember the book"--was all Harriet's answer,
              and spoken with a degree of grave displeasure which Emma thought might
              be safely left to itself.  She, therefore, said no more for some time."""]

In [5]:
# Access the string within the list
document = document[0]

# Tokenize the string
tokens = word_tokenize(document)

# Lowercase each word after tokenizing:
tokens = [word.lower() for word in tokens]

### **3. Building vocab**

In [6]:
# build vocab
vocab = {'<unk>': 0}

for token in Counter(tokens).keys():
  if token not in vocab:
    vocab[token] = len(vocab)

vocab

{'<unk>': 0,
 'emma': 1,
 'woodhouse': 2,
 ',': 3,
 'handsome': 4,
 'clever': 5,
 'and': 6,
 'rich': 7,
 'with': 8,
 'a': 9,
 'comfortable': 10,
 'home': 11,
 'happy': 12,
 'disposition': 13,
 'seemed': 14,
 'to': 15,
 'unite': 16,
 'some': 17,
 'of': 18,
 'the': 19,
 'best': 20,
 'blessings': 21,
 'existence': 22,
 ';': 23,
 'had': 24,
 'lived': 25,
 'nearly': 26,
 'twenty-one': 27,
 'years': 28,
 'in': 29,
 'world': 30,
 'very': 31,
 'little': 32,
 'distress': 33,
 'or': 34,
 'vex': 35,
 'her': 36,
 '.': 37,
 'she': 38,
 'was': 39,
 'youngest': 40,
 'two': 41,
 'daughters': 42,
 'most': 43,
 'affectionate': 44,
 'indulgent': 45,
 'father': 46,
 'consequence': 47,
 'sister': 48,
 "'s": 49,
 'marriage': 50,
 'been': 51,
 'mistress': 52,
 'his': 53,
 'house': 54,
 'from': 55,
 'early': 56,
 'period': 57,
 'mother': 58,
 'died': 59,
 'too': 60,
 'long': 61,
 'ago': 62,
 'for': 63,
 'have': 64,
 'more': 65,
 'than': 66,
 'an': 67,
 'indistinct': 68,
 'remembrance': 69,
 'caresses': 70,
 '

In [7]:
len(vocab)

1814

In [8]:
# extract sentences from data
input_sentences = document.split('\n')

### **4. Converting text to numerical indices**

In [9]:
def text_to_indices(sentence, vocab):
  numerical_sentence = []

  for token in sentence:
    if token in vocab:
      numerical_sentence.append(vocab[token])
    else:
      numerical_sentence.append(vocab['<unk>'])

  return numerical_sentence

In [10]:
input_numerical_sentences = []

for sentence in input_sentences:
  input_numerical_sentences.append(text_to_indices(word_tokenize(sentence.lower()), vocab))

In [11]:
len(input_numerical_sentences)

1000

In [12]:
training_sequence = []

for sentence in input_numerical_sentences:

  for i in range(1, len(sentence)):
    training_sequence.append(sentence[:i+1])

In [13]:
len(training_sequence)

10660

In [14]:
training_sequence[:5]

[[1, 2], [1, 2, 3], [1, 2, 3, 4], [1, 2, 3, 4, 3], [1, 2, 3, 4, 3, 5]]

In [15]:
len_list = []

for sequence in training_sequence:
  len_list.append(len(sequence))

max(len_list)

21

### **5. Applying padding for all training sequences**

In [16]:
padded_training_sequence = []

for sequence in training_sequence:
  padded_training_sequence.append([0]*(max(len_list) - len(sequence)) + sequence)

In [17]:
len(padded_training_sequence[0])

21

In [18]:
padded_training_sequence = torch.tensor(padded_training_sequence, dtype=torch.long)

In [19]:
padded_training_sequence

tensor([[  0,   0,   0,  ...,   0,   1,   2],
        [  0,   0,   0,  ...,   1,   2,   3],
        [  0,   0,   0,  ...,   2,   3,   4],
        ...,
        [  0,   0,   0,  ...,  65,  63,  17],
        [  0,   0,   0,  ...,  63,  17, 354],
        [  0,   0,   0,  ...,  17, 354,  37]])

In [20]:
X = padded_training_sequence[:, :-1]
y = padded_training_sequence[:, -1]

In [21]:
X

tensor([[  0,   0,   0,  ...,   0,   0,   1],
        [  0,   0,   0,  ...,   0,   1,   2],
        [  0,   0,   0,  ...,   1,   2,   3],
        ...,
        [  0,   0,   0,  ..., 191,  65,  63],
        [  0,   0,   0,  ...,  65,  63,  17],
        [  0,   0,   0,  ...,  63,  17, 354]])

In [22]:
y

tensor([  2,   3,   4,  ...,  17, 354,  37])

### **6. Creating Custom Dataset**

In [23]:
class CustomDataset(Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, index):
    return self.X[index], self.y[index]

In [24]:
dataset = CustomDataset(X, y)

In [25]:
len(dataset)

10660

In [26]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

### **7. Building the Model Architecture**

In [27]:
class LSTMModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, 100)
    self.lstm = nn.LSTM(100, 150, batch_first=True)
    self.fc = nn.Linear(150, vocab_size)

  def forward(self, x):
    embedded = self.embedding(x)
    intermediate_hidden_states, (final_hidden_state, final_cell_state) = self.lstm(embedded)
    output = self.fc(final_hidden_state.squeeze(0))

    return output

In [28]:
model = LSTMModel(len(vocab))

### **8. Taking my model to GPU**

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

LSTMModel(
  (embedding): Embedding(1814, 100)
  (lstm): LSTM(100, 150, batch_first=True)
  (fc): Linear(in_features=150, out_features=1814, bias=True)
)

### **9. Setting up parameters, loss and optimizer**

In [30]:
epochs = 1000
learning_rate = 0.0001

In [31]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### **10. Implementing the training loop**

In [32]:
# training loop

for epoch in range(epochs):
  total_loss = 0

  for batch_x, batch_y in dataloader:
    batch_x, batch_y = batch_x.to(device), batch_y.to(device)

    optimizer.zero_grad()

    output = model(batch_x)

    loss = criterion(output, batch_y)

    loss.backward()

    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:.4f}")

Epoch: 1, Loss: 2255.4703
Epoch: 2, Loss: 1947.5474
Epoch: 3, Loss: 1928.3260
Epoch: 4, Loss: 1913.7557
Epoch: 5, Loss: 1898.0533
Epoch: 6, Loss: 1882.3993
Epoch: 7, Loss: 1866.0381
Epoch: 8, Loss: 1848.5788
Epoch: 9, Loss: 1831.7245
Epoch: 10, Loss: 1815.8766
Epoch: 11, Loss: 1798.4972
Epoch: 12, Loss: 1781.5249
Epoch: 13, Loss: 1766.5104
Epoch: 14, Loss: 1748.0876
Epoch: 15, Loss: 1730.5117
Epoch: 16, Loss: 1713.2495
Epoch: 17, Loss: 1697.8657
Epoch: 18, Loss: 1682.8226
Epoch: 19, Loss: 1665.7541
Epoch: 20, Loss: 1650.2064
Epoch: 21, Loss: 1635.6707
Epoch: 22, Loss: 1618.9291
Epoch: 23, Loss: 1603.4690
Epoch: 24, Loss: 1587.1202
Epoch: 25, Loss: 1572.4934
Epoch: 26, Loss: 1555.6903
Epoch: 27, Loss: 1541.7409
Epoch: 28, Loss: 1527.7371
Epoch: 29, Loss: 1514.5154
Epoch: 30, Loss: 1497.8931
Epoch: 31, Loss: 1484.8336
Epoch: 32, Loss: 1471.0156
Epoch: 33, Loss: 1457.0537
Epoch: 34, Loss: 1442.8300
Epoch: 35, Loss: 1429.4336
Epoch: 36, Loss: 1415.9345
Epoch: 37, Loss: 1401.7483
Epoch: 38,

### **11. Model Prediction**

In [38]:
# prediction

def prediction(model, vocab, text):

  # tokenize
  tokenized_text = word_tokenize(text.lower())

  # text -> numerical indices
  numerical_text = text_to_indices(tokenized_text, vocab)

  # padding
  padded_text = torch.tensor([0] * (23 - len(numerical_text)) + numerical_text, dtype=torch.long).unsqueeze(0)

  # Move padded_text to the same device as the model
  padded_text = padded_text.to(device) # The fix

  # send to model
  output = model(padded_text)

  # predicted index
  value, index = torch.max(output, dim=1)

  # merge with text
  return text + " " + list(vocab.keys())[index]

In [41]:
prediction(model, vocab, " Emma spared")

' Emma spared no'

In [42]:
import time

num_tokens = 10
input_text = " Emma spared"

for i in range(num_tokens):
  output_text = prediction(model, vocab, input_text)
  print(output_text)
  input_text = output_text
  time.sleep(0.5)

 Emma spared no
 Emma spared no exertions
 Emma spared no exertions to
 Emma spared no exertions to maintain
 Emma spared no exertions to maintain this
 Emma spared no exertions to maintain this happier
 Emma spared no exertions to maintain this happier flow
 Emma spared no exertions to maintain this happier flow of
 Emma spared no exertions to maintain this happier flow of ideas
 Emma spared no exertions to maintain this happier flow of ideas ,
